In [1]:
import pandas as pd
from datasets import load_from_disk
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
ds = load_from_disk('../data/efcamdat_dataset')
model = AutoModelForSequenceClassification.from_pretrained('../bin/efcamdat_nli_1').to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained('../bin/efcamdat_nli_1')

In [3]:
label_names = ds['test'].features['labels'].names

id2label = {idnum: label for idnum, label in enumerate(label_names)}
label2id = {label: idnum for idnum, label in enumerate(label_names)}

model.config.id2label = id2label
model.config.label2id = label2id

In [4]:
from transformers_interpret import SequenceClassificationExplainer
cls_explainer = SequenceClassificationExplainer(model, tokenizer)

In [5]:
df = pd.read_parquet('../data/efcamdat_shatz_distro.parquet').set_index('writing_id')

In [6]:
def get_topic(example):
    ind = example['writing_id']
    info = df.loc[ind]
    topic = info['topic']
    l1 = info['l1']
    return {'topic': topic, 'l1': l1}

ds['test'] = ds['test'].map(get_topic)

  0%|          | 0/17711 [00:00<?, ?ex/s]

In [18]:
print(df.topic.value_counts().to_string())

topics_to_keep = [
    'Taking inventory in the office',
    'Giving instructions to play a game',
    'Describing routines',
    'Making notes for a visitor',
    'Planning for the future',
    'Planning to attend a music festival',
    'Labeling photos from a safari',
    'Making a dinner party menu',
]

topics_to_exclude = [
    'Introducing yourself by email',
    'Introducing yourself by email',
    'Writing an online profile',
    'Updating your online profile',
]

Introducing yourself by email              34282
Writing about what you do                  33860
Writing an online profile                  29088
Describing your family in an email         24841
Updating your online profile               23546
Describing your favorite day               18765
Writing a party invitation                 18653
Taking inventory in the office             18200
Signing up for a dating website            17686
Giving instructions to play a game         17553
Writing a birthday invitation              16393
Describing routines                        16037
Writing a weather guide for your city      15363
Buying clothes from a catalog              15218
Making notes for a visitor                 14677
Replying to a new penpal                   14450
Planning for the future                    14433
Planning to attend a music festival        13662
Writing a description of your family       13126
Telling someone what you're doing          12652
Writing about what y

In [17]:
def topic_filter(example):
    if example['topic'] in topics_to_keep:
        return True
    else:
        return False
    

for sample in ds['test'].filter(topic_filter).shuffle().select(range(30)):
    word_attributions = cls_explainer(sample['text'], class_name=id2label[sample['labels']])
    print(sample['writing_id'], id2label[sample['labels']])
    print(sample['topic'])
    html = cls_explainer.visualize()

  0%|          | 0/18 [00:00<?, ?ba/s]

64403 Russian
Giving instructions to play a game


414472 Russian
Giving instructions to play a game


243673 Mandarin
Making notes for a visitor


300590 Spanish
Describing your favorite day


...

594316 Portuguese
Making notes for a visitor


1159256 Spanish
Taking inventory in the office


1194383 Arabic
Describing routines


285551 Mandarin
Giving instructions to play a game


989478 Spanish
Describing routines
